# Redes de Computadores
### Computer Networks

# Lab class #2
## Stream oriented network programming with TCP sockets


## Summary

+ Client/Server model
+ Java example
+ Exercise: File transfer

## Client/Server model

+ Two autonomous components:
 - **server** - first to run and usually always running;
 
 - **client** - usually started by user to request a service...
 

Therefore, TCP follows the same abstract client/server model that characterizes UDP communication.

## Client/Server model
### Execution diagram

<img src="https://docs.google.com/drawings/d/e/2PACX-1vRO-wtMt-_EBYjecn_Hwc5xLkBDfrUZQcnqimzpbXJ-YSiaOfyh6U3oQEOLobk5requzhiiwz7d5SlF/pub?w=960&amp;h=720" width="75%">

## TCP channel

* **Reliable** connection between two processes;<br>

* **Initiated** by the client, addressed to a machine and port, where the server **already** awaits connection requests;<br>

* Enables a continuous and bidirectional flow of data (bytes) between the two processes;<br>

* Closed at the request of **either** the client or the server;<br>

* Both the client and server **are aware** when a connection is established or broken/closed.<br>





## TCP oriented communication

+ Communication is based on **sockets**
    - TCP sockets for accepting connections (server)
    - TCP sockets for data exchange (client and server)
    

+ Addresses identify connection endpoints (*origin* and *destination* processes/sockets);
    - **Host** (IP Address)
      ``10.1.233.67, 127.0.0.1, 192.168.1.1, etc.``
    - **Port** (16 bits)
     ``80, 443, 8000, 8080, etc.``
 

TCP and UDP ports are independent. 

We can have a TCP server listening on port 8000 and
an UDP server also on port 8000. These will be different ports.

## Java Programming with UDP datagrams

+ Java package

    [java.net](https://docs.oracle.com/en/java/javase/16/docs/api/java.base/java/net/package-summary.html)
    
    [java.io](https://docs.oracle.com/en/java/javase/16/docs/api/java.base/java/io/package-summary.html)
 
 
+ Java classes

    [Socket](https://docs.oracle.com/en/java/javase/16/docs/api/java.base/java/net/Socket.html)

    [ServerSocket](https://docs.oracle.com/en/java/javase/16/docs/api/java.base/java/net/ServerSocket.html)

    [InputStream](https://docs.oracle.com/en/java/javase/16/docs/api/java.base/java/io/InputStream.html) /
    [OutputStream](https://docs.oracle.com/en/java/javase/16/docs/api/java.base/java/io/OutputStream.html)


## Example: Echo service

+ Client connects to a server (IP + port) and sends some bytes;
+ Server copies that data back to the client.

## Server

In [ ]:
import java.io.*;
import java.net.*;

static final int PORT = 8000;
static final int BUF_SIZE = 1024;

try (ServerSocket ss = new ServerSocket(PORT)) {
    for (;;) {
        try( Socket cs = ss.accept()){
            
            InputStream is = cs.getInputStream();
            OutputStream os = cs.getOutputStream();

            byte[] buf = new byte[BUF_SIZE];

            int n;
            while( (n = is.read( buf )) > 0 )
                os.write( buf, 0, n);
            
        } catch( IOException x) {
            x.printStackTrace();
        }
    }
} catch (IOException x) {
    x.printStackTrace();
}

## Client

In [ ]:
String host = args[0];
String message = args[2];
int port = Integer.valueOf(args[1]);

try (Socket cs = new Socket(host, port)) {
    
    OutputStream os = cs.getOutputStream();
    
    os.write( message.getBytes() );
    cs.shutdownOutput();

    InputStream is = cs.getInputStream();
    
    byte[] echo = is.readAllBytes();
    
    System.out.printf("echo reply: '%s'\n", new String( echo ) );
} catch (IOException x) {
    x.printStackTrace();
}

## Class [ServerSocket](https://docs.oracle.com/en/java/javase/16/docs/api/java.base/java/net/ServerSocket.html)

The ServerSocket class is used **exclusively** in the server to listen for incoming tcp connections.

In [ ]:
// Server
try(ServerSocket socket = new ServerSocket( PORT )) {
    
}

## Class [Socket](https://docs.oracle.com/en/java/javase/16/docs/api/java.base/java/net/Socket.html)

Socket class object represent the actual tcp connections between client and server processes.

Data exchange (read/write of bytes) is supported by the [InputStream](https://docs.oracle.com/en/java/javase/16/docs/api/java.base/java/io/InputStream.html) / [OutputStream](https://docs.oracle.com/en/java/javase/16/docs/api/java.base/java/io/OutputStream.html) objects associated with each Socket instance pair.



In [ ]:
// Client
try(Socket socket = new Socket(host, port)) {
    ...
}

// Server
try(Socket socket = serverSocket.accept()) {
    ...
}

## Exercise 1

Try the provided EchoService sample code locally in your machine, then
using Docker.

1. Study the [tcp.EchoServer](https://github.com/smduarte/RC2021-labs/blob/main/aula2/RC2021-aula2/src/tcp/EchoServer.java) and [tcp.EchoClient](https://github.com/smduarte/RC2021-labs/blob/main/aula2/RC2021-aula2/src/tcp/EchoClient.java) classes. Use the provided [Eclipse project](https://github.com/smduarte/RC2021-labs/blob/main/aula2/RC2021-aula2.zip), by importing it into an Eclipse workspace.


2. Run the sample code locally in your machine...

    + Compile the classes, either in Eclipse or using javac;
    + Launch the server (no params)
    + Run the echo client, using either "localhost" or "127.0.0.1" for
    the ip address of the server;
    

3. Run the sample code using Docker...

    + Build the docker image (assuming the classes are compiled), running in the project root folder (containing the [Dockerfile](https://github.com/smduarte/RC2021-labs/blob/main/aula2/RC2021-aula2/Dockerfile)):
    
    `docker build . -t rc2021-aula2`<br><br>
    
    + Launch the echo server:
    
    `docker run -ti --name echoserver_tcp rc2021-aula2 java tcp.EchoServer`<br><br>

    + Inspect the container to determine its IP address:
    
    `docker inspect echoserver_tcp`<br><br>
    
    + Run the echo client, replacing `<ip>` with the ip address of the echo server. 
    
    `docker run -ti rc2021-aula2 java tcp.EchoClient <ip> 8000 "this is a test"`

## Exercise 2 - TCP file transfer

Use TCP to transfer a file between a client and a server.

Complete the provided [server](https://github.com/smduarte/RC2021-labs/blob/main/aula2/RC2021-aula2-ex2.zip), knowing that the client first
sends a line of text with the name of the file being transferred, then followed by the bytes of the file's contents.

The server should consider the transfer is complete when the client closes the connection.